In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_path = os.path.join(os.getcwd(),'..','..')
sys.path.append(project_path)

from utils.transformation import reusable

## **DimUser**

**AUTOLOADER**

In [0]:
df_user = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimUser/chekpoint")\
        .option("schemaEvolutionMode","addNewColumn")\
        .load("abfss://bronzelouis@louisxxxxproject.dfs.core.windows.net/DimUser")\
        .repartition(4)



In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))
df_user

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])

In [0]:
df_user.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimUser/chekpoint")\
            .trigger(once=True)\
            .option("path", "abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimUser/data")\
            .toTable("spotify_cata.silver.DimUser")

##**DimArtist**

In [0]:
df_art = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimArtist/chekpoint")\
        .option("schemaEvolutionMode","addNewColumn")\
        .load("abfss://bronzelouis@louisxxxxproject.dfs.core.windows.net/DimArtist")\
        .repartition(4)


In [0]:
df_art_obj = reusable()

df_art = df_art_obj.dropColumns(df_art, ['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])



In [0]:
# WARNING: This will delete all streaming state and progress for DimArtist!
dbutils.fs.rm("abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimArtist/chekpoint", True)

In [0]:
df_art.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimArtist/chekpoint")\
            .trigger(once=True)\
            .option("path","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimArtist/data")\
            .toTable("spotify_cata.silver.DimArtist")#load data into catalog silver

##**DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimTrack/chekpoint")\
        .option("schemaEvolutionMode","addNewColumn")\
        .load("abfss://bronzelouis@louisxxxxproject.dfs.core.windows.net/DimTrack")\
        .repartition(4)



In [0]:
# WARNING: This will delete all streaming state and progress for DimArtist!
dbutils.fs.rm("abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimTrack/chekpoint", True)

In [0]:
df_track = df_track.withColumn("durationFlag", when(col('duration_sec')<150,"low")
                                               .when(col('duration_sec')<300,"medium")
                                               .otherwise("high"))

df_track = df_track.withColumn("track_name", regexp_replace(col("track_name"),'-',' '))
df_track = reusable().dropColumns(df_track,['_rescued_data'])

In [0]:
df_track.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimTrack/chekpoint")\
            .trigger(once=True)\
            .option("path","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimTrack/data")\
            .toTable("spotify_cata.silver.DimTrack")#load data into catalog silver

##**DimDate**

In [0]:
df_date = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimDate/chekpoint")\
        .option("schemaEvolutionMode","addNewColumn")\
        .load("abfss://bronzelouis@louisxxxxproject.dfs.core.windows.net/DimDate")\
        .repartition(4)



In [0]:
df_date = reusable().dropColumns(df_date, ["_rescued_data"])

In [0]:
df_date.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimDate/chekpoint")\
            .trigger(once=True)\
            .option("path","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/DimDate/data")\
            .toTable("spotify_cata.silver.DimDate")#load data into catalog silver

%md
##**FactStream**

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/FactStream/chekpoint")\
        .option("schemaEvolutionMode","addNewColumn")\
        .load("abfss://bronzelouis@louisxxxxproject.dfs.core.windows.net/FactStream")\
        .repartition(4)



In [0]:
df_fact = reusable().dropColumns(df_fact,['_rescued_data'])

df_fact.writeStream.format("delta")\
            .outputMode("append")\
            .option("checkpointLocation","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/FactStream/chekpoint")\
            .trigger(once=True)\
            .option("path","abfss://silverlouis@louisxxxxproject.dfs.core.windows.net/FactStream/data")\
            .toTable("spotify_cata.silver.FactStream")#load data into catalog silver
        